# Prerequisite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jul 20 07:23:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install same version of library as Kaggle Notebook

In [ ]:
!cp -f /content/drive/MyDrive/kaggle/commonlit/roberta-large/requirements.txt ./
!cat ./requirements.txt

#pandas==1.2.3
sklearn==0.0
sklearn-pandas==2.1.0
torch==1.7.0
torchmetrics==0.2.0
#torchtext==0.8.0a0+cd6902d
torchvision==0.8.1
transformers==4.5.1
datasets==1.9.0
accelerate==0.3.0

In [ ]:
!pip uninstall -r ./requirements.txt -y

Found existing installation: sklearn 0.0
Uninstalling sklearn-0.0:
  Successfully uninstalled sklearn-0.0
Found existing installation: sklearn-pandas 1.8.0
Uninstalling sklearn-pandas-1.8.0:
  Successfully uninstalled sklearn-pandas-1.8.0
Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102


In [ ]:
!pip install -r ./requirements.txt 

     |████████████████████████████████| 776.7 MB 4.1 kB/s 
     |████████████████████████████████| 176 kB 89.8 MB/s 
     |████████████████████████████████| 12.7 MB 72.9 MB/s 
     |████████████████████████████████| 2.1 MB 70.0 MB/s 
     |████████████████████████████████| 262 kB 84.4 MB/s 
     |████████████████████████████████| 49 kB 8.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 28.5 MB 1.2 MB/s 
     |████████████████████████████████| 3.3 MB 79.8 MB/s 
     |████████████████████████████████| 895 kB 70.0 MB/s 
     |████████████████████████████████| 118 kB 78.6 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 243 kB 86.7 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=0c6b2cdac3c378b810263e0098b8e03f779f7ee9367d019ade9de25e27909877
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaa

In [ ]:
!pip freeze |grep -e random -e math -e numpy -e pandas -e torch -e transformers -e sklearn -e gc

mpmath==1.2.1
numpy==1.19.5
pandas==1.1.5
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
sklearn==0.0
sklearn-pandas==2.1.0
tensorflow-gcs-config==2.5.0
torch==1.7.0
torchmetrics==0.2.0
torchsummary==1.5.1
torchtext==0.10.0
torchvision==0.8.1
transformers==4.5.1


## Prepare dataset

### kaggle.json

In [ ]:
!mkdir -p /root/.kaggle/
!cp ./drive/MyDrive/kaggle/commonlit/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

### Competition dataset

In [ ]:
!mkdir -p ../input/commonlitreadabilityprize/
!kaggle competitions download -c commonlitreadabilityprize -p ../input/commonlitreadabilityprize/
!cp -f ./drive/MyDrive/kaggle/commonlit/train_stratiKfold.csv.zip ../input/commonlitreadabilityprize/

  0% 0.00/6.79k [00:00<?, ?B/s]
100% 6.79k/6.79k [00:00<00:00, 6.47MB/s]
  0% 0.00/108 [00:00<?, ?B/s]
100% 108/108 [00:00<00:00, 102kB/s]
  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 76.7MB/s]


In [ ]:
!unzip -o ../input/commonlitreadabilityprize/train.csv.zip -d ../input/commonlitreadabilityprize/
!unzip -o ../input/commonlitreadabilityprize/train_stratiKfold.csv.zip -d ../input/commonlitreadabilityprize/

Archive:  ../input/commonlitreadabilityprize/train.csv.zip
  inflating: ../input/commonlitreadabilityprize/train.csv  
Archive:  ../input/commonlitreadabilityprize/train_stratiKfold.csv.zip
  inflating: ../input/commonlitreadabilityprize/train_stratiKfold.csv  


In [ ]:
!ls ../input/commonlitreadabilityprize/

sample_submission.csv  train.csv      train_stratiKfold.csv
test.csv	       train.csv.zip  train_stratiKfold.csv.zip


### Pretrained RoBERTa Large 
- Pretrain RoBERTa Large in the same way as this notebook
  - https://www.kaggle.com/maunish/clrp-pytorch-roberta-pretrain
- Dataset:
  - https://www.kaggle.com/iamnishipy/roberta-large-20210712191259-mlm

In [ ]:
!mkdir -p ../input/commonlitreadabilityprize/pretrained-model/
!kaggle datasets download iamnishipy/roberta-large-20210712191259-mlm

100% 8.52G/8.53G [01:56<00:00, 55.9MB/s]
100% 8.53G/8.53G [01:56<00:00, 78.6MB/s]


In [ ]:
!unzip -o ./roberta-large-20210712191259-mlm.zip -d ../input/commonlitreadabilityprize/pretrained-model/

Archive:  ./roberta-large-20210712191259-mlm.zip
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/merges.txt  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/pytorch_model.bin  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/special_tokens_map.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/tokenizer_config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/training_args.bin  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large/vocab.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large_chk/checkpoint-1200/config.json  
  inflating: ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large_chk/checkpoint-1200/optimizer.p

In [ ]:
!ls ../input/commonlitreadabilityprize/pretrained-model/

clrp_roberta_large  clrp_roberta_large_chk  text.txt


# Overview
This is kernel is almost the same as [Lightweight Roberta solution in PyTorch](https://www.kaggle.com/andretugan/lightweight-roberta-solution-in-pytorch), but instead of "roberta-base", it starts from [Maunish's pre-trained model](https://www.kaggle.com/maunish/clrp-roberta-base).

Acknowledgments: some ideas were taken from kernels by [Torch](https://www.kaggle.com/rhtsingh) and [Maunish](https://www.kaggle.com/maunish).

In [ ]:
#!pip install transformers accelerate datasets

In [ ]:
import os
import math
import random
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import KFold

import gc
gc.enable()

## Prepare dataset

In [ ]:
NUM_FOLDS = 5
NUM_EPOCHS = 3
BATCH_SIZE = 8
MAX_LEN = 248
EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
ROBERTA_PATH = "../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large"
TOKENIZER_PATH = "../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large"
# ROBERTA_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
# TOKENIZER_PATH = "../input/clrp-roberta-base/clrp_roberta_base"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

In [ ]:
train_df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

# Remove incomplete entries if any.
train_df.drop(train_df[(train_df.target == 0) & (train_df.standard_error == 0)].index,
              inplace=True)
train_df.reset_index(drop=True, inplace=True)

test_df = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission_df = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

# Dataset

In [ ]:
class LitDataset(Dataset):
    def __init__(self, df, inference_only=False):
        super().__init__()

        self.df = df        
        self.inference_only = inference_only
        self.text = df.excerpt.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.inference_only:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)        
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return (input_ids, attention_mask)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, target)

# Model
The model is inspired by the one from [Maunish](https://www.kaggle.com/maunish/clrp-roberta-svm).

In [ ]:
class LitModel(nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(ROBERTA_PATH)
        config.update({"output_hidden_states":True, 
                       "hidden_dropout_prob": 0.0,
                       "layer_norm_eps": 1e-7})                       
        
        self.roberta = AutoModel.from_pretrained(ROBERTA_PATH, config=config)
        #https://towardsdatascience.com/attention-based-deep-multiple-instance-learning-1bb3df857e24
        # 768: node fully connected layer
        # 512: node attention layer
        # self.attention = nn.Sequential(            
        #     nn.Linear(768, 512),            
        #     nn.Tanh(),                       
        #     nn.Linear(512, 1),
        #     nn.Softmax(dim=1)
        # )        

        # self.regressor = nn.Sequential(                        
        #     nn.Linear(768, 1)                        
        # )


        #768 -> 1024
        #512 -> 768
        self.attention = nn.Sequential(            
            nn.Linear(1024, 768),            
            nn.Tanh(),                       
            nn.Linear(768, 1),
            nn.Softmax(dim=1)
        )        

        self.regressor = nn.Sequential(                        
            nn.Linear(1024, 1)                        
        )
        

    def forward(self, input_ids, attention_mask):
        roberta_output = self.roberta(input_ids=input_ids,
                                      attention_mask=attention_mask)        

        # There are a total of 13 layers of hidden states.
        # 1 for the embedding layer, and 12 for the 12 Roberta layers.
        # We take the hidden states from the last Roberta layer.
        last_layer_hidden_states = roberta_output.hidden_states[-1]

        # The number of cells is MAX_LEN.
        # The size of the hidden state of each cell is 768 (for roberta-base).
        # In order to condense hidden states of all cells to a context vector,
        # we compute a weighted average of the hidden states of all cells.
        # We compute the weight of each cell, using the attention neural network.
        weights = self.attention(last_layer_hidden_states)
                
        # weights.shape is BATCH_SIZE x MAX_LEN x 1
        # last_layer_hidden_states.shape is BATCH_SIZE x MAX_LEN x 768        
        # Now we compute context_vector as the weighted average.
        # context_vector.shape is BATCH_SIZE x 768
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1)        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(context_vector)

In [ ]:
def eval_mse(model, data_loader):
    """Evaluates the mean squared error of the |model| on |data_loader|"""
    model.eval()            
    mse_sum = 0

    with torch.no_grad():
        for batch_num, (input_ids, attention_mask, target) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)                        
            target = target.to(DEVICE)           
            
            pred = model(input_ids, attention_mask)                       

            mse_sum += nn.MSELoss(reduction="sum")(pred.flatten(), target).item()
                

    return mse_sum / len(data_loader.dataset)

In [ ]:
def predict(model, data_loader):
    """Returns an np.array with predictions of the |model| on |data_loader|"""
    model.eval()

    result = np.zeros(len(data_loader.dataset))    
    index = 0
    
    with torch.no_grad():
        for batch_num, (input_ids, attention_mask) in enumerate(data_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
                        
            pred = model(input_ids, attention_mask)                        

            result[index : index + pred.shape[0]] = pred.flatten().to("cpu")
            index += pred.shape[0]

    return result

In [ ]:
def train(model, model_path, train_loader, val_loader,
          optimizer, scheduler=None, num_epochs=NUM_EPOCHS):    
    best_val_rmse = None
    best_epoch = 0
    step = 0
    last_eval_step = 0
    eval_period = EVAL_SCHEDULE[0][1]    

    start = time.time()

    for epoch in range(num_epochs):                           
        val_rmse = None         

        for batch_num, (input_ids, attention_mask, target) in enumerate(train_loader):
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)            
            target = target.to(DEVICE)                        

            optimizer.zero_grad()
            
            model.train()

            pred = model(input_ids, attention_mask)
                                                        
            mse = nn.MSELoss(reduction="mean")(pred.flatten(), target)
                        
            mse.backward()

            optimizer.step()
            if scheduler:
                scheduler.step()
            
            if step >= last_eval_step + eval_period:
                # Evaluate the model on val_loader.
                elapsed_seconds = time.time() - start
                num_steps = step - last_eval_step
                print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                last_eval_step = step
                
                val_rmse = math.sqrt(eval_mse(model, val_loader))                            

                print(f"Epoch: {epoch} batch_num: {batch_num}", 
                      f"val_rmse: {val_rmse:0.4}")

                for rmse, period in EVAL_SCHEDULE:
                    if val_rmse >= rmse:
                        eval_period = period
                        break                               
                
                if not best_val_rmse or val_rmse < best_val_rmse:                    
                    best_val_rmse = val_rmse
                    best_epoch = epoch
                    torch.save(model.state_dict(), model_path)
                    print(f"New best_val_rmse: {best_val_rmse:0.4}")
                else:       
                    print(f"Still best_val_rmse: {best_val_rmse:0.4}",
                          f"(from epoch {best_epoch})")                                    
                    
                start = time.time()
                                            
            step += 1
                        
    
    return best_val_rmse

In [ ]:
#怪しい
def create_optimizer(model):
    #model.named_parameters():
    #Base -> 205
    #Large -> 397
    named_parameters = list(model.named_parameters())    

    #Base
    # roberta_parameters = named_parameters[:197]    
    # attention_parameters = named_parameters[199:203]
    # regressor_parameters = named_parameters[203:]
    
    #Large
    roberta_parameters = named_parameters[:389]    
    attention_parameters = named_parameters[391:395]
    regressor_parameters = named_parameters[395:]
        
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]

    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})

    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if "bias" in name else 0.01
        lr = 2e-5
        #roberta-base: 
        # if layer_num >= 69: #4/12layers       
        #     lr = 5e-5
        # if layer_num >= 1f33: #8/12layers
        #     lr = 1e-4
        #roberta-large
        if layer_num >= 133: #8/24layers     
            lr = 5e-5
        if layer_num >= 261: #16/24layers
            lr = 1e-4


        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})

    return AdamW(parameters)

In [ ]:
# SEED = 1000
# kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)
# for fold, (train_indices, val_indices) in enumerate(kfold.split(train_df)):
#     print(fold)
#     print('------------')
#     print(val_indices)

# Debug

In [ ]:
model = LitModel().to(DEVICE)

Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
named_parameters = list(model.named_parameters())
len(named_parameters)

397

In [ ]:
# for name, param in model.named_parameters():
#     print(name)

# Train


In [ ]:
gc.collect()

SEED = 1000
list_val_rmse = []

kfold = KFold(n_splits=NUM_FOLDS, random_state=SEED, shuffle=True)

for fold, (train_indices, val_indices) in enumerate(kfold.split(train_df)):    
    print(f"\nFold {fold + 1}/{NUM_FOLDS}")
    model_path = f"model_{fold + 1}.pth"
        
    set_random_seed(SEED + fold)
    
    train_dataset = LitDataset(train_df.loc[train_indices])    
    val_dataset = LitDataset(train_df.loc[val_indices])    
        
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              drop_last=True, shuffle=True, num_workers=2)    
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=2)    
        
    set_random_seed(SEED + fold)    
    
    model = LitModel().to(DEVICE)
    
    optimizer = create_optimizer(model)                        
    # scheduler = get_cosine_schedule_with_warmup(
    #     optimizer,
    #     num_training_steps=NUM_EPOCHS * len(train_loader),
    #     num_warmup_steps=50)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_training_steps=NUM_EPOCHS * len(train_loader),
        num_warmup_steps=200)
    
    list_val_rmse.append(train(model, model_path, train_loader,
                               val_loader, optimizer, scheduler=scheduler))

    del model
    gc.collect()
    
    print("\nPerformance estimates:")
    print(list_val_rmse)
    print("Mean:", np.array(list_val_rmse).mean())
    


Fold 1/5


Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



16 steps took 12.7 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9948
New best_val_rmse: 0.9948

16 steps took 11.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7868
New best_val_rmse: 0.7868

16 steps took 11.8 seconds
Epoch: 0 batch_num: 48 val_rmse: 1.03
Still best_val_rmse: 0.7868 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7324
New best_val_rmse: 0.7324

16 steps took 11.8 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6477
New best_val_rmse: 0.6477

16 steps took 11.7 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7103
Still best_val_rmse: 0.6477 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.593
New best_val_rmse: 0.593

16 steps took 11.8 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.8043
Still best_val_rmse: 0.593 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6618
Still best_val_rmse: 0.593 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 160 val_rmse: 0.5791
New best_val_rmse: 0

Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



16 steps took 12.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.095
New best_val_rmse: 1.095

16 steps took 11.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.9736
New best_val_rmse: 0.9736

16 steps took 11.8 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7386
New best_val_rmse: 0.7386

16 steps took 11.8 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7665
Still best_val_rmse: 0.7386 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6394
New best_val_rmse: 0.6394

16 steps took 11.8 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6582
Still best_val_rmse: 0.6394 (from epoch 0)

16 steps took 11.7 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7236
Still best_val_rmse: 0.6394 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6049
New best_val_rmse: 0.6049

16 steps took 11.8 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6561
Still best_val_rmse: 0.6049 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 160 val_rmse: 0.621
Still best_val_rm

Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



16 steps took 12.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.084
New best_val_rmse: 1.084

16 steps took 11.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.005
New best_val_rmse: 1.005

16 steps took 11.8 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.8805
New best_val_rmse: 0.8805

16 steps took 11.8 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.8542
New best_val_rmse: 0.8542

16 steps took 11.8 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6538
New best_val_rmse: 0.6538

16 steps took 11.8 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7449
Still best_val_rmse: 0.6538 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7316
Still best_val_rmse: 0.6538 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.7962
Still best_val_rmse: 0.6538 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.653
New best_val_rmse: 0.653

16 steps took 11.8 seconds
Epoch: 0 batch_num: 160 val_rmse: 0.6611
Still best_val_rmse: 0.653 (from epoc

Some weights of RobertaModel were not initialized from the model checkpoint at ../input/commonlitreadabilityprize/pretrained-model/clrp_roberta_large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



16 steps took 12.6 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.127
New best_val_rmse: 1.127

16 steps took 11.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.823
New best_val_rmse: 0.823

16 steps took 11.8 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.759
New best_val_rmse: 0.759

16 steps took 11.8 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6436
New best_val_rmse: 0.6436

16 steps took 11.8 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6383
New best_val_rmse: 0.6383

16 steps took 11.8 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6689
Still best_val_rmse: 0.6383 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7232
Still best_val_rmse: 0.6383 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6925
Still best_val_rmse: 0.6383 (from epoch 0)

16 steps took 11.8 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6663
Still best_val_rmse: 0.6383 (from epoch 0)

16 steps took 11.8 seconds


# Inference

In [ ]:
test_dataset = LitDataset(test_df, inference_only=True)

In [ ]:
all_predictions = np.zeros((len(list_val_rmse), len(test_df)))

test_dataset = LitDataset(test_df, inference_only=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         drop_last=False, shuffle=False, num_workers=2)

for index in range(len(list_val_rmse)):            
    model_path = f"model_{index + 1}.pth"
    print(f"\nUsing {model_path}")
                        
    model = LitModel()
    model.load_state_dict(torch.load(model_path))    
    model.to(DEVICE)
    
    all_predictions[index] = predict(model, test_loader)
    
    del model
    gc.collect()

In [ ]:
predictions = all_predictions.mean(axis=0)
submission_df.target = predictions
print(submission_df)
#submission_df.to_csv("submission.csv", index=False)

## Upload model

In [ ]:
!mkdir -p ./output/
!cp -f ./model* ./output/
#CHANGEME
!cp -f ./drive/MyDrive/kaggle/commonlit/pretrained-roberta-base/dataset-metadata.json ./output/dataset-metadata.json
!sed -i -e "s/roberta-base/roberta-large-`TZ=JST-9 date +"%Y%m%d%H%M%S"`-sch/" ./output/dataset-metadata.json
!sed -i -e "s/Roberta-base/Roberta-large-`TZ=JST-9 date +"%m%d%H%M%S"`-sch/" ./output/dataset-metadata.json
!kaggle datasets create -p ./output/

In [ ]:
!cat ./output/dataset-metadata.json